In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from math import radians, sin, cos, acos
import sys
import re
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from time import time
import datetime
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, recall_score
from sklearn.metrics import confusion_matrix as CM
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve as ROC
from sklearn.metrics import accuracy_score as AC
from sklearn.metrics import f1_score

In [25]:
import warnings
warnings.filterwarnings('ignore')

In [26]:
data_path=r'C:\Codefield\CODE_C\PYTHON\ML\data.xlsx'
df=pd.read_excel(data_path)
df.head()

,Time Stamp,T,Po,P,Pa,U,DD,Ff,ff10,ff3,...,Cm,Ch,VV,Td,RRR,tR,E,Tg,E',sss
0,24.12.2023 23:00,-2.8,775.6,776.1,-0.2,85,无风,0,NaN,NaN,...,NaN,NaN,2.8,-5.0,1,12.0,NaN,NaN,NaN,NaN
1,24.12.2023 20:00,-1.3,775.8,776.2,0.1,72,从西南方吹来的风,1,NaN,8.0,...,NaN,NaN,3.1,-5.7,1,12.0,NaN,NaN,NaN,NaN
2,24.12.2023 17:00,4.6,775.7,776.2,-0.2,41,从西方吹来的风,2,NaN,8.0,...,NaN,NaN,8.0,-7.5,1,12.0,NaN,NaN,NaN,NaN
3,24.12.2023 14:00,5.9,775.9,776.3,-2.1,37,从西方吹来的风,4,NaN,8.0,...,NaN,NaN,9.0,-7.7,1,12.0,NaN,NaN,NaN,NaN
4,24.12.2023 11:00,4.0,778.0,778.5,-0.3,44,从西北偏西方向吹来的风,3,NaN,7.0,...,NaN,NaN,10.0,-7.4,1,12.0,NaN,NaN,NaN,NaN


In [27]:
df.loc[df['VV']=='低于 0.1','VV']=0
df['VV']=df['VV'].astype('float64')
""" df['Time']=df["Time Stamp"].apply(lambda x:x[11:13])
df.drop(df[df['Time']!='11'].index,inplace=True) """
df.drop(columns=["Time Stamp","tR"],inplace=True)
attributes=df.columns.values
discrete=["DD", "WW", "W2", "H"]
continuous=["T", "Po", "P", "Pa", "U", "Ff", "ff3", "Tn", "Tx", "VV", "Td"]
for attribute in attributes:
    if attribute not in ["T", "Po", "P", "Pa", "U", "Ff", "ff3", "Tn", "Tx", "VV", "Td","DD", "WW", "W2", "H","RRR"]:
        df.drop(columns=[attribute],inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2832 entries, 0 to 2831
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   T       2832 non-null   float64
 1   Po      2832 non-null   float64
 2   P       2831 non-null   float64
 3   Pa      2654 non-null   float64
 4   U       2832 non-null   int64  
 5   DD      2832 non-null   object 
 6   Ff      2832 non-null   int64  
 7   ff3     2518 non-null   float64
 8   WW      2832 non-null   object 
 9   W2      2047 non-null   object 
 10  Tn      2751 non-null   float64
 11  Tx      2566 non-null   float64
 12  H       2582 non-null   object 
 13  VV      2831 non-null   float64
 14  Td      2832 non-null   float64
 15  RRR     2832 non-null   int64  
dtypes: float64(9), int64(3), object(4)
memory usage: 354.1+ KB


In [28]:
x0 =np.array(df.iloc[:, :-1])
y0 =np.array(df.iloc[:, -1])
length=15
num=int(len(y0)/8)
x=np.zeros((num,length*8))
x = x.astype(object)
y=np.zeros(num)
for i in range(0,num):
    y[i]=y0[i*8]
    for j in range(0,8):
        x[i,15*j:15*j+15]=x0[8*i+j,:]
x=pd.DataFrame(x)
y=pd.DataFrame(y)
print(x.shape) 
print(y.shape) 
print(y.value_counts())
print(x.head())

(354, 120)
(354, 1)
1.0    211
0.0    143
Name: count, dtype: int64
   0      1      2    3   4            5   6     7                    8    \
0 -2.8  775.6  776.1 -0.2  85           无风   0   NaN                   霾    
1 -2.5  778.2  778.6  0.3  63  从西南偏西方向吹来的风   1   4.0  云量发展情况没有进行观测或无法观测。    
2 -3.1  781.6  782.0  0.3  63  从西南偏西方向吹来的风   2  10.0  云量发展情况没有进行观测或无法观测。    
3 -4.4  781.3  781.7  0.5  49      从西方吹来的风   3   NaN  云量发展情况没有进行观测或无法观测。    
4 -1.5  777.3  777.7  1.4  52  从西北偏西方向吹来的风   4  11.0  云量发展情况没有进行观测或无法观测。    

   9    ...          110 111   112                  113                   114  \
0  NaN  ...  从西北偏西方向吹来的风   2   5.0  云量发展情况没有进行观测或无法观测。   在整个相应周期内云遮住了一半或以下的天空   
1  NaN  ...      从西方吹来的风   2   8.0  云量发展情况没有进行观测或无法观测。   在整个相应周期内云遮住了一半或以下的天空   
2  NaN  ...      从西方吹来的风   3   NaN  云量发展情况没有进行观测或无法观测。   在整个相应周期内云遮住了一半或以下的天空   
3  NaN  ...  从西北偏西方向吹来的风   3  11.0  云量发展情况没有进行观测或无法观测。   在整个相应周期内云遮住了一半或以下的天空   
4  NaN  ...     从西北方吹来的风   4  10.0                   霾   在整个相应周期

In [29]:
xtrain, xtest, y_train, y_test = x.copy(deep=True),x.copy(deep=True),y.copy(deep=True),y.copy(deep=True)
for i in [xtrain, xtest, y_train, y_test]:
    i.index = range(i.shape[0])
y_label_count = y_train.value_counts()
print(y_label_count)

1.0    211
0.0    143
Name: count, dtype: int64


In [30]:

#实例化，填补策略为“mean”均值
continuous=[]
for i in range(0,8):
    for j in range(0,15):
        if j in [5,8,9,12]:
            continue
        continuous.append(15*i+j)
impmean = SimpleImputer(missing_values = np.nan, strategy="mean").fit(xtrain.loc[:, continuous])
xtrain.loc[:, continuous] = impmean.transform(xtrain.loc[:, continuous])
xtest.loc[:, continuous] = impmean.transform(xtest.loc[:, continuous])
ss = StandardScaler().fit(xtrain.loc[:, continuous])
xtrain.loc[:, continuous] = ss.transform(xtrain.loc[:, continuous])
xtest.loc[:, continuous] = ss.transform(xtest.loc[:, continuous])
print(xtrain.head())

        0         1         2         3         4            5         6    \
0 -1.917693  1.902425  1.911243 -0.354657  0.597897           无风 -1.592102   
1 -1.888855  2.272519  2.266115  0.263193 -0.947034  从西南偏西方向吹来的风 -0.884057   
2 -1.946531  2.756489  2.748741  0.263193 -0.947034  从西南偏西方向吹来的风 -0.176011   
3 -2.071495  2.713786  2.706157  0.510333 -1.930172      从西方吹来的风  0.532034   
4 -1.792729   2.14441  2.138361  1.622462   -1.7195  从西北偏西方向吹来的风  1.240079   

        7                    8    9    ...          110       111       112  \
0       0.0                   霾   NaN  ...  从西北偏西方向吹来的风 -0.173844 -0.999095   
1 -1.481021  云量发展情况没有进行观测或无法观测。   NaN  ...      从西方吹来的风 -0.173844  0.205348   
2  1.095715  云量发展情况没有进行观测或无法观测。   NaN  ...      从西方吹来的风  0.509941       0.0   
3       0.0  云量发展情况没有进行观测或无法观测。   NaN  ...  从西北偏西方向吹来的风  0.509941   1.40979   
4  1.525171  云量发展情况没有进行观测或无法观测。   NaN  ...     从西北方吹来的风  1.193727  1.008309   

                   113                   114       115  

In [31]:
discrete=[]
for i in range(0,8):
    for j in [5,8,9,12]:
        discrete.append(15*i+j)
si = SimpleImputer(missing_values=np.nan, strategy="most_frequent").fit(xtrain.loc[:, discrete])
#然后我们用训练集中的众数来同时填补训练集和测试集
xtrain.loc[:, discrete] = si.transform(xtrain.loc[:, discrete])
xtest.loc[:, discrete] = si.transform(xtest.loc[:, discrete])
xtrain.loc[:, discrete].isnull().mean()
xtest.loc[:, discrete].isnull().mean()
#处理分类变量：将分类型变量编码
oe = OrdinalEncoder().fit(xtrain.loc[:, discrete])
#用训练集的编码结果来编码训练和测试矩阵
xtrain.loc[:,  discrete] = oe.transform(xtrain.loc[:, discrete])
xtest.loc[:, discrete] = oe.transform(xtest.loc[:, discrete])


In [32]:
xtrain=xtrain.astype('float64')
xtest=xtest.astype('float64')

In [33]:
times = time()
for kernel in ["linear", "poly", "rbf", "sigmoid"]:
    clf = SVC(kernel=kernel, gamma="auto",C=10, degree=2,cache_size=5000, class_weight = 'balanced').fit(xtrain, y_train)
    result = clf.predict(xtest)
    score = clf.score(xtest, y_test)
    recall = recall_score(y_test, result)
    auc = roc_auc_score(y_test, clf.decision_function(xtest))
    macro=f1_score(y_true=y_test, y_pred=result, average="macro")
    mat=CM(y_test,result)
    print("%s 's testing accuracy %f, recall is %f', auc is %f,macro is %f" %(kernel, score, recall, auc,macro))
    print(mat)
    print(datetime.datetime.fromtimestamp(time()-times).strftime("%M:%S:%f"))


linear 's testing accuracy 0.949153, recall is 0.947867', auc is 0.973089,macro is 0.947434
[[136   7]
 [ 11 200]]
00:05:630780
poly 's testing accuracy 1.000000, recall is 1.000000', auc is 1.000000,macro is 1.000000
[[143   0]
 [  0 211]]
00:05:811460
rbf 's testing accuracy 1.000000, recall is 1.000000', auc is 1.000000,macro is 1.000000
[[143   0]
 [  0 211]]
00:06:005424
sigmoid 's testing accuracy 0.570621, recall is 0.568720', auc is 0.616843,macro is 0.565616
[[ 82  61]
 [ 91 120]]
00:06:143501
